In [1]:
import numpy as np
import math

In [2]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
 

betahat = s 
sebetahat = np.repeat(sigma,s.shape[0])
 

In [57]:
def autoselect_scales_mix_norm(betahat, sebetahat, max_class=None, mult=2):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = 2*np.sqrt(np.max(betahat**2 - sebetahat**2))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
        
    
    return out
     
def autoselect_scales_mix_exp(betahat, sebetahat, max_class=None , mult=1.5,tt=1.5):
    sigmaamin = np.min(sebetahat) / 10
    if np.all(betahat**2 < sigmaamin**2):  # Fix the typo and ensure logical comparison
        sigmaamax = 8 * sigmaamin
    else:
        sigmaamax = tt*np.sqrt(np.max(betahat**2  ))
    
    if mult == 0:
        out = np.array([0, sigmaamax / 2])
    else:
        npoint = math.ceil(math.log2(sigmaamax / sigmaamin) / math.log2(mult))

        # Generate the sequence (-npoint):0 using np.arange
        sequence = np.arange(-npoint, 1)

        # Calculate the output
        out = np.concatenate(([0], (1/mult) ** (-sequence) * sigmaamax))
        if max_class!=None:
            # Check if the length of out is equal to max_class
            if len(out) != max_class:
            # Generate a sequence from min(out) to max(out) with length max_class
                out = np.linspace(np.min(out), np.max(out), num=max_class)
                if(out[2] <1e-2 ):
                 out[2: ] <- out[2: ] +1e-2
         
    
    return out

    
    

In [58]:
class cebnm:
    def __init__(self,
                 mixture_type="normal",
                 betahat=None,
                 sebetahat=None,
                 max_class=None,
                 mult=None,
                 tt=1.5):
        self.type = mixture_type

        if betahat is not None and sebetahat is not None:
            self.betahat   = betahat
            self.sebetahat = sebetahat
            
            # Common arguments
            args = {
                'betahat': betahat,
                'sebetahat': sebetahat
            }
            
            # Optional arguments
            if mult is not None:
                args['mult'] = mult
            if max_class is not None:
                args['max_class'] = max_class

            # Call the appropriate function based on the mixture type
            if self.type == "normal":
                self.scale = autoselect_scales_mix_norm(**args)
            elif self.type == "exp":
                args['tt'] = tt  # Add 'tt' for "exp" type
                self.scale =autoselect_scales_mix_exp(**args)
            ncomp = self.scale.shape[0]
            self.mixture_prop = np.full(ncomp, 1/ncomp)


In [59]:
betahat=  np.array([1,2,3,4,5])
sebetahat=np.array([1,0.4,5,1,1])

tamer=cebnm(betahat=betahat,
            sebetahat=sebetahat)
print(tamer.type)
print(tamer.scale)
print(tamer.mixture_prop)

normal
[0.         0.03827328 0.07654655 0.15309311 0.30618622 0.61237244
 1.22474487 2.44948974 4.89897949 9.79795897]


In [22]:
print(autoselect_scales_mix_norm ( np.array([1,2,3,4,5]),  np.array([1,0.4,5,1,1]) ))
print(autoselect_scales_mix_exp ( np.array([1,2,3,4,5]),  np.array([1,0.4,5,1,1])))




[0.         0.03827328 0.07654655 0.15309311 0.30618622 0.61237244
 1.22474487 2.44948974 4.89897949 9.79795897]
[0.         0.03853673 0.0578051  0.08670765 0.13006147 0.19509221
 0.29263832 0.43895748 0.65843621 0.98765432 1.48148148 2.22222222
 3.33333333 5.         7.5       ]


In [54]:
n = betahat.shape[0]  # Get the length of betahat
array = np.full(n, 1/n)